#Table Extraction of pdf_1

In [1]:
#importing necessary libraries
import os
import pdfplumber
import re
import pandas as pd
#namedtuple allows you to create simple classes with named fields, making code more readable and self-documenting
from collections import namedtuple

In [2]:
#creating a namedtuple with class head
head = namedtuple('head', 'Line Item Description UOM Qty Unit_Net_Price Extended_Price')

In [3]:
#reading the pdf
invoice = "/content/Pdf_1.pdf"

In [4]:
invoice

'/content/Pdf_1.pdf'

### "ocrmypdf" is an external command-line tool used for adding OCR (Optical Character Recognition) text to PDF files. It takes an input PDF file, processes it to recognize text in images, and outputs a new PDF file with the recognized text.

In [18]:
os.system(f'ocrmypdf {invoice} output.pdf')

0

###Using the pdfplumber library to open the pdf file, read the first page,extract the text from the first page and printing the extracted text

In [6]:
with pdfplumber.open('output.pdf') as pdf:
  page = pdf.pages[0]
  text = page.extract_text(x_tolerance=2)
  print(text)

INTEGRA‘ Invoice Number : 26762745
LIMIT JNCERTAINT © me Customer Number invoice Date Page
Integra LifeSciences Sales LLC 33078 30-DE C-21 1 0f 2
1100 Campus Road, Princeton, NJ 08540 United States
Phone (609) 275-0500 Fax (888)-980-7742
Remit By = NTEGRA LIFESCIENCES Remit By INTEGRA LIFESCIENCES
Check: P.O. BOX 404129 Wire/ACH: BANK OF AMERICA
ATLANTA GA 30384-4129 100 WEST 33RD STREET, NY, NY, 10001
ABA 111000012, SWIFT BOFAUS3N
ACCOUNT 3756327238
Bill To: Attn: Accounts Payable Ship To: NATUS NEUROLOGY
NATUS NEUROLOGY 3150 PLEASANT VIEW RD
12301 LAKE UNDERHILL RD STE 201 MIDDLETON WI 53562-4800
ORLANDO FL 32828-4511
Tracking: UPS/1Z 724 251 03 5206 0749
Invoice Due Invoice
Customer PO Payment Terms Ship To PO Date Currency Incoterm Sales Order
57265 Net 45 13-FEB-22 USD FOB ORIGIN 4582446
Unit Net
Line Item Description UOM Qty Price Extended Price
1 NT8501376 Open-Ended Peritoneal Catheter w/Slits, 120cm EA 13 39.86 518.18
Lot# 5961218(13)
2 NT8501229 Portnoy Flanged Ventricular Ca

In [7]:
#splitting the extracted text
lines = text.split('\n')

In [8]:
lines

['INTEGRA‘ Invoice Number : 26762745',
 'LIMIT JNCERTAINT © me Customer Number invoice Date Page',
 'Integra LifeSciences Sales LLC 33078 30-DE C-21 1 0f 2',
 '1100 Campus Road, Princeton, NJ 08540 United States',
 'Phone (609) 275-0500 Fax (888)-980-7742',
 'Remit By = NTEGRA LIFESCIENCES Remit By INTEGRA LIFESCIENCES',
 'Check: P.O. BOX 404129 Wire/ACH: BANK OF AMERICA',
 'ATLANTA GA 30384-4129 100 WEST 33RD STREET, NY, NY, 10001',
 'ABA 111000012, SWIFT BOFAUS3N',
 'ACCOUNT 3756327238',
 'Bill To: Attn: Accounts Payable Ship To: NATUS NEUROLOGY',
 'NATUS NEUROLOGY 3150 PLEASANT VIEW RD',
 '12301 LAKE UNDERHILL RD STE 201 MIDDLETON WI 53562-4800',
 'ORLANDO FL 32828-4511',
 'Tracking: UPS/1Z 724 251 03 5206 0749',
 'Invoice Due Invoice',
 'Customer PO Payment Terms Ship To PO Date Currency Incoterm Sales Order',
 '57265 Net 45 13-FEB-22 USD FOB ORIGIN 4582446',
 'Unit Net',
 'Line Item Description UOM Qty Price Extended Price',
 '1 NT8501376 Open-Ended Peritoneal Catheter w/Slits, 12

In [9]:
#creating the regular expression to search for the data
col_re = re.compile(r'^(\d{1}) ([A-Z0-9]+) (.+?) (\w+) (\d+) (\d+\.\d{2}) (\d+\.\d{2})')

In [10]:
#creating another regular expression
desc_col_re = re.compile(r'Lot#\s+(\d+)\((\d+)\)')

In [13]:
#By matching and searching the regular expressions, the wanted data is stored in a list with the use of namedtuple
col_data=[]
desc_data=[]
for line in text.split('\n'):
  if desc_col_re.match(line):
     desc_data+=line.split('\n')
     #desc_data

  line = col_re.search(line)
  if line:
     Line = line.group(1)
     Item = line.group(2)
     Des = line.group(3)
     UOM = line.group(4)
     Qty = line.group(5)
     Unit_Net_Price = line.group(6)
     Extended_Price = line.group(7)

     if desc_data:
                Des += ' ' + desc_data.pop(0)


     col_data.append(head(Line, Item, Des, UOM, Qty, Unit_Net_Price, Extended_Price))

In [14]:
desc_data

['Lot# 5961226(3)']

In [17]:
#converting to dataframe and listing the data
df=pd.DataFrame(col_data)
df

,Line,Item,Description,UOM,Qty,Unit_Net_Price,Extended_Price
0,1,NT8501376,"Open-Ended Peritoneal Catheter w/Slits, 120cm",EA,13,39.86,518.18
1,2,NT8501229,"Portnoy Flanged Ventricular Catheter, 18cm Lot...",EA,8,29.69,237.52
2,3,NT8500112,Natus Multi-Purpose Valve wAnti Siphon Device ...,EA,1,128.97,128.97
3,4,NT97001S01,Luer Connector Lot# 5905603(1),EA,5,3.01,15.05
4,5,NT8501195,FOLTZ FLUSHING RESERVOIR Lot# 5949500(5),EA,1,34.14,34.14
5,6,NT83024S01,STERILE VENTRICULAR CATHETER GUIDE,EA,11,14.68,161.48
6,7,NT8501391,"Pudenz Peritoneal Catheter Barium Imp Stripe, ...",EA,3,25.95,77.85
